In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')  # Don't open a browser window
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)

driver.get("https://nation.africa/kenya")
print("Page Title:", driver.title)

driver.quit()


Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


Page Title: Attention Required! | Cloudflare


In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import random

# Some real browser-like user agents
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.5938.149 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4 Safari/605.1.15",
]

options = Options()
options.add_argument(f"user-agent={random.choice(user_agents)}")
# Run visible browser to avoid headless detection
# options.add_argument('--headless')  # 👈 turn OFF headless for now
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Remove navigator.webdriver flag (tells site it's a bot)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Open Nation page
driver.get("https://nation.africa/kenya")
print("Page Title:", driver.title)

driver.quit()


Page Title: Nation - Breaking News, Kenya, Africa, Politics, Business, Sports | HOME | Daily Nation


In [12]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import random

# Setup browser with stealth
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64)...",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4)...",
]
options = Options()
options.add_argument(f"user-agent={random.choice(user_agents)}")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Visit Nation homepage
driver.get("https://nation.africa/kenya")
time.sleep(3)

soup = BeautifulSoup(driver.page_source, 'html.parser')
article_blocks = soup.find_all("a", class_="story-link")  # may need adjusting

titles, urls, dates, contents, sources = [], [], [], [], []

for article in article_blocks[:10]:  # scrape 10 for test
    try:
        title = article.get_text(strip=True)
        url = article.get('href')
        if not url.startswith("http"):
            url = "https://nation.africa" + url

        # Visit article page
        driver.get(url)
        time.sleep(3)
        article_soup = BeautifulSoup(driver.page_source, 'html.parser')

        date = article_soup.find("time", class_="date")
        date_text = date.get_text(strip=True) if date else "N/A"

        paragraphs = article_soup.select("div.text-block.blk-txt div.paragraph-wrapper p")
        content_text = " ".join([p.get_text(strip=True) for p in paragraphs])

        titles.append(title)
        urls.append(url)
        dates.append(date_text)
        contents.append(content_text)
        sources.append("Nation")

    except Exception as e:
        print(f"Error: {e}")
        continue

driver.quit()

# Save to DataFrame
df = pd.DataFrame({
    "title": titles,
    "url": urls,
    "date": dates,
    "content": contents,
    "source": sources
})

df.to_csv("nation_articles.csv", index=False)
print("✅ Done. Saved as nation_articles.csv")


✅ Done. Saved as nation_articles.csv


In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random

options = Options()
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4)",
]
options.add_argument(f"user-agent={random.choice(user_agents)}")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

driver.get("https://nation.africa/kenya")
time.sleep(3)

soup = BeautifulSoup(driver.page_source, "html.parser")
all_links = soup.find_all("a", href=True)

titles, urls = [], []

for link in all_links:
    href = link['href']
    if href.startswith("/kenya") and link.find("h3"):  # ensures it's a proper article block
        url = "https://nation.africa" + href
        title = link.get_text(strip=True)
        urls.append(url)
        titles.append(title)

driver.quit()

# Confirm what we captured
for t, u in zip(titles[:5], urls[:5]):
    print(f"📰 {t}\n🔗 {u}\n")


📰 Kuria resigns as Ruto’s senior adviserThe former Gatundu South MP could be setting the stage for a comeback in the political arena.News4 hours ago
🔗 https://nation.africa/kenya/news/moses-kuria-resigns-as-president-ruto-s-senior-adviser--5110976

📰 Opposition leaders call for boycott of Kenya Kwanza-linked businessesNews2 hours ago
🔗 https://nation.africa/kenya/news/opposition-leaders-call-for-boycott-of-kenya-kwanza-linked-businesses-5110774

📰 Embu protests enter second day over MP Gitonga Mukunji’s detentionEmbu10 hours ago
🔗 https://nation.africa/kenya/counties/embu/embu-protests-enter-second-day-over-mp-gitonga-mukunji-s-detention-5110188

📰 PREMIUMCourt freezes firm's bank account after failing to deliver goldAfriswiss Commodities Ltd was to deliver 25 kilos of gold to a Dubai firm.News9 hours agoListen
🔗 https://nation.africa/kenya/news/court-freezes-firm-s-bank-account-after-failing-to-deliver-gold--5110608

📰 PREMIUMOjwang widow’s burial rites reignites Luo customs debate'I 

In [14]:
def scrape_nation_article(url, driver):
    try:
        driver.get(url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Date
        time_tag = soup.find("time", class_="date")
        date = time_tag.get_text(strip=True) if time_tag else "N/A"

        # Content
        content_div = soup.find("div", class_="text-block blk-txt")
        if content_div:
            paragraphs = content_div.find_all("p")
            content = " ".join(p.get_text(strip=True) for p in paragraphs)
        else:
            content = "N/A"

        return date, content

    except Exception as e:
        print(f"❌ Error scraping {url}: {e}")
        return "N/A", "N/A"


In [15]:
from tqdm import tqdm
import pandas as pd

options.add_argument(f"user-agent={random.choice(user_agents)}")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

data = []

for title, url in tqdm(zip(titles, urls), total=len(urls)):
    date, content = scrape_nation_article(url, driver)
    data.append({
        "title": title,
        "url": url,
        "date": date,
        "content": content,
        "source": "Nation",
        "label": "real"  # We'll update this later
    })

driver.quit()

# Save to CSV
df = pd.DataFrame(data)
df.to_csv("NationArticles.csv", index=False)
print("✅ Done! Articles saved.")


  5%|▌         | 4/79 [01:03<13:46, 11.02s/it]

❌ Error scraping https://nation.africa/kenya/news/court-freezes-firm-s-bank-account-after-failing-to-deliver-gold--5110608: Message: target frame detached
  (failed to check if window was closed: disconnected: unable to send message to renderer)
  (Session info: chrome=138.0.7204.97)
Stacktrace:
	GetHandleVerifier [0x0xe744a3+62419]
	GetHandleVerifier [0x0xe744e4+62484]
	(No symbol) [0x0xcb1f90]
	(No symbol) [0x0xca2b80]
	(No symbol) [0x0xca1cf1]
	(No symbol) [0x0xcbf912]
	(No symbol) [0x0xd25d6c]
	(No symbol) [0x0xd40159]
	(No symbol) [0x0xd1f266]
	(No symbol) [0x0xcee852]
	(No symbol) [0x0xcef6f4]
	GetHandleVerifier [0x0x10e4793+2619075]
	GetHandleVerifier [0x0x10dfbaa+2599642]
	GetHandleVerifier [0x0xe9b04a+221050]
	GetHandleVerifier [0x0xe8b2c8+156152]
	GetHandleVerifier [0x0xe91c7d+183213]
	GetHandleVerifier [0x0xe7c388+94904]
	GetHandleVerifier [0x0xe7c512+95298]
	GetHandleVerifier [0x0xe6766a+9626]
	BaseThreadInitThunk [0x0x76b3fcc9+25]
	RtlGetAppContainerNamedObjectPath [0x0x77

  6%|▋         | 5/79 [01:04<09:52,  8.01s/it]

❌ Error scraping https://nation.africa/kenya/counties/homa-bay/ojwang-widow-s-burial-rites-omission-reignites-luo-customs-debate-5106868: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=138.0.7204.97)
Stacktrace:
	GetHandleVerifier [0x0xe744a3+62419]
	GetHandleVerifier [0x0xe744e4+62484]
	(No symbol) [0x0xcb2133]
	(No symbol) [0x0xca1b40]
	(No symbol) [0x0xcbf912]
	(No symbol) [0x0xd25d6c]
	(No symbol) [0x0xd40159]
	(No symbol) [0x0xd1f266]
	(No symbol) [0x0xcee852]
	(No symbol) [0x0xcef6f4]
	GetHandleVerifier [0x0x10e4793+2619075]
	GetHandleVerifier [0x0x10dfbaa+2599642]
	GetHandleVerifier [0x0xe9b04a+221050]
	GetHandleVerifier [0x0xe8b2c8+156152]
	GetHandleVerifier [0x0xe91c7d+183213]
	GetHandleVerifier [0x0xe7c388+94904]
	GetHandleVerifier [0x0xe7c512+95298]
	GetHandleVerifier [0x0xe6766a+9626]
	BaseThreadInitThunk [0x0x76b3fcc9+25]
	RtlGetAppContainerNamedObjectPath [0x0x771f8

 23%|██▎       | 18/79 [01:04<04:02,  3.97s/it]

...

 66%|██████▌   | 52/79 [01:04<00:52,  1.95s/it]


❌ Error scraping https://nation.africa/kenya/counties/relief-in-kilifi-as-nuclear-plant-project-relocated-to-siaya-5109306: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x0xe744a3+62419]
	GetHandleVerifier [0x0xe744e4+62484]
	(No symbol) [0x0xcb1f90]
	(No symbol) [0x0xcedb48]
	(No symbol) [0x0xd1f326]
	(No symbol) [0x0xd1af15]
	(No symbol) [0x0xd1a496]
	(No symbol) [0x0xc83a45]
	(No symbol) [0x0xc83f9e]
	(No symbol) [0x0xc8442d]
	GetHandleVerifier [0x0x10e4793+2619075]
	GetHandleVerifier [0x0x10dfbaa+2599642]
	GetHandleVerifier [0x0xe9b04a+221050]
	GetHandleVerifier [0x0xe8b2c8+156152]
	GetHandleVerifier [0x0xe91c7d+183213]
	(No symbol) [0x0xc83710]
	(No symbol) [0x0xc82f1d]
	GetHandleVerifier [0x0x1218aec+3881500]
	BaseThreadInitThunk [0x0x76b3fcc9+25]
	RtlGetAppContainerNamedObjectPath [0x0x771f82ae+286]
	RtlGetAppContainerNamedObjectPath [0x0x771f827e+238]

❌ Error scraping https://nation.africa/kenya/counties/mombasa/saba-saba-mombasa-forgotten-crossroads-of-resista

100%|██████████| 79/79 [01:05<00:00,  1.22it/s]


❌ Error scraping https://nation.africa/kenya/business/tears-for-sweetness-the-plight-of-cane-farmers-5108118: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x0xe744a3+62419]
	GetHandleVerifier [0x0xe744e4+62484]
	(No symbol) [0x0xcb1f90]
	(No symbol) [0x0xcedb48]
	(No symbol) [0x0xd1f326]
	(No symbol) [0x0xd1af15]
	(No symbol) [0x0xd1a496]
	(No symbol) [0x0xc83a45]
	(No symbol) [0x0xc83f9e]
	(No symbol) [0x0xc8442d]
	GetHandleVerifier [0x0x10e4793+2619075]
	GetHandleVerifier [0x0x10dfbaa+2599642]
	GetHandleVerifier [0x0xe9b04a+221050]
	GetHandleVerifier [0x0xe8b2c8+156152]
	GetHandleVerifier [0x0xe91c7d+183213]
	(No symbol) [0x0xc83710]
	(No symbol) [0x0xc82f1d]
	GetHandleVerifier [0x0x1218aec+3881500]
	BaseThreadInitThunk [0x0x76b3fcc9+25]
	RtlGetAppContainerNamedObjectPath [0x0x771f82ae+286]
	RtlGetAppContainerNamedObjectPath [0x0x771f827e+238]

❌ Error scraping https://nation.africa/kenya/business/fines-interest-on-unpaid-bills-rises-by-sh4-1-billion--5108236: Message

✅ Done! Articles saved.
